In [1]:
import pandas as pd
import numpy as np
import os,glob
import requests
import json
from bs4 import BeautifulSoup
import xlsxwriter

In [2]:
os.chdir("C:/Users/Ganesh P Umarani/Desktop/Optum/pm/Medline")

In [3]:
# Upload the file with weblinks to product
medline=pd.read_excel('Medline_sourcing_done.xlsx')

In [4]:
medline.columns

Index(['Secondary_Key', 'Link Owner's', 'DISTRIBUTOR',
       'DISTRIBUTOR PRODUCT NUMBER', 'Manufacturer', 'mnf_sourced ',
       'Mfg_part_num', 'mpn_sourced', 'Product Description', 'Cluster Iter4',
       'Cluster ID', 'Weblink', 'Sourcing Status'],
      dtype='object')

In [5]:
# remove the rows where there are no weblinks
medline=medline[medline.Weblink.notnull()]

In [6]:
# medline_sample=medline.iloc[0:2,:]

In [7]:
#declare the output dataframe to store the scrapped results
output=pd.DataFrame()

In [8]:
#declare a dataframe to store urls which ran into issues so that ther can be looked upon later
issue_url=pd.DataFrame(columns=['Weblinks'])

In [10]:
#loop through each row of the uploaded file and get the weblink to send the request
for idx,row in medline.iterrows():
    url=row['Weblink'].strip()
    try:
        data=requests.get(url) #Send request and capture response
    except:
        print("Issue with  {}".format(url))
        issue_url=issue_url.append({'Weblinks':url},ignore_index=True)
        continue
    
    #check if the status returened on sending request is 200 which means to check if we got any response
    if(data.status_code==200):
        data_pretty=BeautifulSoup(data.text,'html.parser') #Parse the response to html format
        to_append={} # declare a dict to store the values scrapped in key value format, later this dict will be appended to output dataframe
        
        #Below snippet scraps long and short description and also product explaination
        try:
            descriptions=data_pretty.find_all('div',{'class':'medSKUDescription'})
            ldesc=descriptions[0].select('h1')[0].select('a')[0].contents[0]
            sdesc=descriptions[0].select('h4')[0].contents[0]
            prod_exp=descriptions[0].find_all('div',{'class':'medMoreSKUDescText'})[0].contents[0].strip()
            to_append['Long_desc']=ldesc
            to_append['Short_desc']=sdesc
            to_append['Product_details']=prod_exp
        except:
            print("Issue with  {}".format(url))
            issue_url=issue_url.append({'Weblinks':url},ignore_index=True)
            continue
        
        #below snippet will scrape the details of the product which are given in a table
        try:
            detail_table=data_pretty.find_all('div',{'class':'medSKUFltRt'})[0].select_one('table',{'class':'medSKUTableDetails'})
            list_of_tr=detail_table.find_all('tr')
            
            for i in range(len(list_of_tr)):
                list_of_td=list_of_tr[i].find_all('td')
                to_append[list_of_td[0].contents[0].strip()]=list_of_td[1].select('span')[0].contents[0].strip()
        except:
            print("Issue with  {}".format(url))
            issue_url=issue_url.append({'Weblinks':url},ignore_index=True)
            continue
        
        #Below snippet will scrape the specification of the product
        try:
            spec_table=data_pretty.find_all('table',{'id':'medSkuSpecTable'})
            list_of_tr_st=spec_table[0].find_all('tr')
            
            for k in range(len(list_of_tr_st)):
                list_of_td_st=list_of_tr_st[k].find_all('td')
                to_append[list_of_td_st[0].contents[0].strip()]=list_of_td_st[1].contents[0].strip()
        except:
            print("Issue with  {}".format(url))
            issue_url=issue_url.append({'Weblinks':url},ignore_index=True)
            continue
        
        # All the columns from the i/p file which are needed in o/p file are included in the dict
        to_append['Secondar_key']=row['Secondary_Key']
        to_append['DISTRIBUTOR']=row['DISTRIBUTOR']
        to_append['DISTRIBUTOR PRODUCT NUMBER']=row['DISTRIBUTOR PRODUCT NUMBER']
        to_append['Manufacturer']=row['Manufacturer']
        to_append['mnf_sourced ']=row['mnf_sourced ']
        to_append['Mfg_part_num']=row['Mfg_part_num']
        to_append['mpn_sourced']=row['mpn_sourced']
        to_append['Product Description']=row['Product Description']
        to_append['Weblink']=row['Weblink']
        
        #Append the dict to the o/p dataframe
        output=output.append(to_append,ignore_index=True)
        print("Output length---> {}".format(len(output)))
    

Output length---> 1
Output length---> 2
